In [1]:
import random
import time

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [3]:
import tensorflow as tf
import numpy as np

np.set_printoptions(precision=4, suppress=True)

In [4]:
import tensorflow.contrib.layers

In [5]:
import gym

In [29]:
env = gym.make('CartPole-v1')
# env = gym.make('LunarLander-v2')

[2017-01-25 21:12:45,257] Making new env: CartPole-v1


In [30]:
def CartPoleQNetwork(state, unused_is_training):
    hidden = tf.contrib.layers.fully_connected(
        state, 32,
        activation_fn=tf.nn.elu,
        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
        scope='hidden1')
    hidden = tf.contrib.layers.fully_connected(
        hidden, 32,
        activation_fn=tf.nn.elu,
        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
        scope='hidden2')

    value = tf.contrib.layers.linear(hidden, 1,
                                     weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                     biases_initializer=tf.constant_initializer(0.),
                                     scope='value')
    adv = tf.contrib.layers.linear(hidden, env.action_space.n,  # num_actions
                                   weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                   scope='advantage')
    adv = tf.sub(adv, tf.reduce_mean(adv, reduction_indices=1, keep_dims=True), 'advantage')

    output = tf.add(value, adv, 'output')
    return output

In [31]:
ROLLOUT_LEN = 20
GAMMA = 0.99

In [32]:
buf = learner.WeightedExperienceBuffer(0.6, 0.4, 100, 1 << 15)
old_s = env.reset()
for _ in range(50):
    ss, aa, rr, ss1, gg = [], [], [], [], []
    done = False
    while not done and len(ss) < ROLLOUT_LEN:
        a = env.action_space.sample()
    
        s, r, done, _ = env.step(a)
        ss.append(old_s)
        aa.append(a)
        rr.append(r)
        ss1.append(s)
        gg.append(GAMMA if not done else 0.)

        old_s = s
    
    rew = 0.
    g = 1.
    for i in reversed(range(len(ss))):
        rew = rr[i] + gg[i] * rew
        g *= gg[i]
        ss1[i] = old_s
        rr[i] = r
        gg[i] = g
    
    if done:
        old_s = env.reset()
    
    buf.add(np.array(ss), np.array(aa), np.array(rr), np.array(ss1), np.array(gg), 100)

In [33]:
import learner
reload(learner)

<module 'learner' from 'learner.pyc'>

In [34]:
sess.close()
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [35]:
with tf.device('/cpu:0'):
    ql = learner.QLearner(buf, CartPoleQNetwork, options={
        'clip_grad': 3.,
        'learning_rate': 0.0001,
        'update_steps': 15000,
        })

INFO:tensorflow:Summary name model/hidden1/weights:0/grad is illegal; using model/hidden1/weights_0/grad instead.


[2017-01-25 21:13:06,545] Summary name model/hidden1/weights:0/grad is illegal; using model/hidden1/weights_0/grad instead.


INFO:tensorflow:Summary name model/hidden1/weights:0 is illegal; using model/hidden1/weights_0 instead.


[2017-01-25 21:13:06,547] Summary name model/hidden1/weights:0 is illegal; using model/hidden1/weights_0 instead.


INFO:tensorflow:Summary name model/hidden1/biases:0/grad is illegal; using model/hidden1/biases_0/grad instead.


...

[2017-01-25 21:13:06,584] Summary name model/advantage/weights:0 is illegal; using model/advantage/weights_0 instead.


INFO:tensorflow:Summary name model/advantage/biases:0/grad is illegal; using model/advantage/biases_0/grad instead.


[2017-01-25 21:13:06,587] Summary name model/advantage/biases:0/grad is illegal; using model/advantage/biases_0/grad instead.


INFO:tensorflow:Summary name model/advantage/biases:0 is illegal; using model/advantage/biases_0 instead.


[2017-01-25 21:13:06,590] Summary name model/advantage/biases:0 is illegal; using model/advantage/biases_0 instead.


In [36]:
ql.Init(sess, 'practice/cartpole-ql-1')

In [ ]:
episode_rew = 0.
episode_len = 0.
old_s = env.reset()
for i in range(100500100):
    ss, aa, rr, ss1, gg = [], [], [], [], []
    done = False
    if i % 1 == 0:
        while not done and len(ss) < ROLLOUT_LEN:
            epsilon = 0.3 / (1 + (ql.cur_step or 0.) / 500000.)
            if np.random.sample() < epsilon:
                a = env.action_space.sample()
            else:
                a = sess.run(ql.act_s1, {ql.state1: np.reshape(old_s, (1, -1)),
                                         ql.is_training: False})
                a = a[0]

            s, r, done, _ = env.step(a)

            ss.append(old_s)
            aa.append(a)
            rr.append(r)
            ss1.append(s)
            gg.append(GAMMA if not done else 0.0)

            episode_rew += r
            episode_len += 1

            old_s = s

        rew = 0.
        g = 1.
        for i in reversed(range(len(ss))):
            rew = rr[i] + gg[i] * rew
            g *= gg[i]
            ss1[i] = old_s
            rr[i] = r
            gg[i] = g

        if done:
            ql.Stat({'Env/Reward': episode_rew, 'Env/Length': episode_len})
            episode_rew, episode_len = 0., 0.
            old_s = env.reset()

        buf.add(np.array(ss), np.array(aa), np.array(rr), np.array(ss1), np.array(gg), 100)

    ql.Step(sess)